In [1]:
# the OLS plot 
### CODE ###
import pandas as pd
from statsmodels.formula.api import ols
import numpy as np
from statsmodels.stats.sandwich_covariance import cov_hac
from math import sqrt
import random
import scipy.stats

df = pd.read_csv('asgn3_data.csv')

In [2]:
arr = []
for i in range(17):
    num = 20 + i*5
    fit = ols('y ~ x1 + x2 + x3', data=df[:num]).fit()
    arr.append(fit)
    with open("regression_outputs.txt", "a") as outf:
        outf.write(f"REGRESSION {i}\n")
        outf.write(fit.summary().as_text())
        outf.write("\n\n")

In [3]:
# create two dataframes to represent the tables for both hypothesis, table0: B1=0, table1: B1 = 0.1
table1 = pd.DataFrame({'t_test':[], 'resampled bootstrap':[], 'wild bootstrap':[]})
table2 = pd.DataFrame({'t_test':[], 'resampled bootstrap':[], 'wild bootstrap':[]})

In [4]:
### P value using a t-test with hypothisis of B1=0
### there should be 96 degrees of freedom = sample size - 3 parameters - 1 intercept
p_values= []
for i in range(17):
    p_values.append(arr[i].pvalues[1])
    
table1.t_test = p_values

p_values = []
for i in range(17):
    p_values.append(arr[i].t_test("x1=0.1").pvalue)

table2.t_test = p_values

In [5]:
# function that returns new values of y and their new disturbances
def new_values_resampled_disturbances(fitted_values, residuals):
    resampled_residuals = []
    si = len(fitted_values)
    for i in range(si):
        resampled_residuals.append(sqrt(si/(si-4))*residuals[i])
    Nvalues = []
    for i in range(si):
        Nvalues.append(fit.params[0]+fit.params[2]*df.x2[i]+fit.params[3]*df.x3[i]+resampled_residuals[random.randint(0,si-1)])
        
    return Nvalues

In [6]:
# create y* and store them in the new_datapoints array
def get_bootstrap_dataframes_resampled(i):
    new_datapoints = []
    B = 99
    for b in range(B):
        new_datapoints.append(new_values_resampled_disturbances(arr[i].fittedvalues, arr[i].resid))

    n = len(arr[i].resid)
    # create new dataframes using the y* values
    new_dataframes = []
    new_dataframes.append(df[:n])
    # there will be B+1 elements in our arrays of dataframes, parameter estimates etc.
    for i in range(B):
        df1 = pd.DataFrame({'y': new_datapoints[i], 'x1':df.x1[:n], 'x2': df.x2[:n], 'x3': df.x3[:n]})
        new_dataframes.append(df1)
        
    return new_dataframes

In [7]:
# in data will be 17*100 dataframes containing all the data required to run the regressions
#### need to change get_bootstrap_dataframe function if doing is for the residuals bootstrap!!
datas = []
for i in range(17):
    datas.append(get_bootstrap_dataframes_resampled(i))

In [8]:
def get_b1_array_for_dataframe_array(a_dataframe_array):
    b1_array = []
    l = len(a_dataframe_array)
    for i in range(l):
        a_fit = ols('y ~ x1 + x2 + x3', data=a_dataframe_array[i]).fit()
        b1_array.append(a_fit.params[1])
        
    return b1_array

In [9]:
def get_b1_error_array_for_dataframe_array(a_dataframe_array):
    b1_error_array = []
    l = len(a_dataframe_array)
    for i in range(l):
        a_fit = ols('y ~ x1 + x2 + x3', data=a_dataframe_array[i]).fit()
        b1_error_array.append(a_fit.bse[1])
        
    return b1_error_array

In [10]:
b1s = []
b1_errors = []
for i in range(17):
    b1s.append(get_b1_array_for_dataframe_array(datas[i]))
    b1_errors.append(get_b1_error_array_for_dataframe_array(datas[i]))

In [11]:
# running t-tests for all elements
all_p_values_0 = []
all_p_values_1 = []
for i in range(17):
    l = len(b1s[i])
    one_p_values_0 = []
    one_p_values_1 = []
    m = arr[i].df_resid
    for j in range(l):
        t = ((b1s[i][j])/b1_errors[i][j])
        one_p_values_0.append(scipy.stats.t.sf(abs(t), df=m)*2)
        t = ((b1s[i][j]-0.1)/b1_errors[i][j])
        one_p_values_1.append(scipy.stats.t.sf(abs(t), df=m)*2)
    
    all_p_values_0.append(one_p_values_0)
    all_p_values_1.append(one_p_values_1)

In [12]:
# indicator function
final_p_value_0 = []
for i in range(17):
    l = len(b1s[i])
    ind1 = 0
    ind2 = 0
    for j in range(l):
        if j == 1:
            continue
        else:
            if all_p_values_0[i][j] > all_p_values_0[i][0]:
                ind2 = ind2 + 1
            else:
                ind1 = ind1 + 1
    
    if ind1>ind2:
        final_p_value_0.append(ind2*2/100)
    else:
        final_p_value_0.append(ind1*2/100)

In [13]:
# indicator function for B1 = 1
final_p_value_1 = []
for i in range(17):
    l = len(b1s[i])
    ind1 = 0
    ind2 = 0
    for j in range(l):
        if j == 1:
            continue
        else:
            if all_p_values_1[i][j] > all_p_values_1[i][0]:
                ind2 = ind2 + 1
            else:
                ind1 = ind1 + 1
    
    if ind1>ind2:
        final_p_value_1.append(ind2*2/100)
    else:
        final_p_value_1.append(ind1*2/100)

In [14]:
table1['resampled bootstrap'] = final_p_value_0
table2['resampled bootstrap'] = final_p_value_1

In [15]:
def new_values_wild(fitted_values, residuals):
    Nvalues = []
    si = len(fitted_values)
    for i in range(si):
        if random.randint(0,1) == 1:
              Nvalues.append(fitted_values[i]+residuals[i])
        else:
            Nvalues.append(fitted_values[i]-residuals[i])
    
    return Nvalues

In [16]:
# create y* and store them in the new_datapoints array
def get_bootstrap_dataframes_wild(i):
    new_datapoints = []
    B = 99
    for b in range(B):
        new_datapoints.append(new_values_wild(arr[i].fittedvalues, arr[i].resid))

    n = len(arr[i].resid)
    # create new dataframes using the y* values
    new_dataframes = []
    new_dataframes.append(df[:n])
    # there will be B+1 elements in our arrays of dataframes, parameter estimates etc.
    for i in range(B):
        df1 = pd.DataFrame({'y': new_datapoints[i], 'x1':df.x1[:n], 'x2': df.x2[:n], 'x3': df.x3[:n]})
        new_dataframes.append(df1)
        
    return new_dataframes

In [17]:
# in data will be 17*100 dataframes containing all the data required to run the regressions
datas = []
for i in range(17):
    datas.append(get_bootstrap_dataframes_wild(i))
    
b1s = []
b1_errors = []
for i in range(17):
    b1s.append(get_b1_array_for_dataframe_array(datas[i]))
    b1_errors.append(get_b1_error_array_for_dataframe_array(datas[i]))

In [18]:
# running t-tests for all elements
all_p_values_0 = []
all_p_values_1 = []
for i in range(17):
    l = len(b1s[i])
    one_p_values_0 = []
    one_p_values_1 = []
    m = arr[i].df_resid
    for j in range(l):
        t = ((b1s[i][j])/b1_errors[i][j])
        one_p_values_0.append(scipy.stats.t.sf(abs(t), df=m)*2)
        t = ((b1s[i][j]-0.1)/b1_errors[i][j])
        one_p_values_1.append(scipy.stats.t.sf(abs(t), df=m)*2)
    
    all_p_values_0.append(one_p_values_0)
    all_p_values_1.append(one_p_values_1)

In [19]:
# indicator function for B1 = 0
final_p_value_0 = []
for i in range(17):
    l = len(b1s[i])
    ind1 = 0
    ind2 = 0
    for j in range(l):
        if j == 1:
            continue
        else:
            if all_p_values_0[i][j] > all_p_values_0[i][0]:
                ind2 = ind2 + 1
            else:
                ind1 = ind1 + 1
    
    if ind1>ind2:
        final_p_value_0.append(ind2*2/100)
    else:
        final_p_value_0.append(ind1*2/100)

In [20]:
# indicator function for B1 = 0.1
final_p_value_1 = []
for i in range(17):
    l = len(b1s[i])
    ind1 = 0
    ind2 = 0
    for j in range(l):
        if j == 1:
            continue
        else:
            if all_p_values_1[i][j] > all_p_values_1[i][0]:
                ind2 = ind2 + 1
            else:
                ind1 = ind1 + 1
    
    if ind1>ind2:
        final_p_value_1.append(ind2*2/(100))
    else:
        final_p_value_1.append(ind1*2/(100))

In [21]:
table1['wild bootstrap'] = final_p_value_0
table2['wild bootstrap'] = final_p_value_1

In [22]:
table1

,t_test,resampled bootstrap,wild bootstrap
0,0.719165,0.60,0.68
1,0.573931,0.86,0.84
2,0.378378,0.78,0.94
3,0.482205,0.96,0.90
4,0.442633,0.98,0.94
5,0.518310,0.78,0.98
6,0.484104,0.90,0.86
7,0.497988,0.90,0.84
8,0.538540,0.96,0.82
9,0.323445,0.54,0.94


In [23]:
table2

,t_test,resampled bootstrap,wild bootstrap
0,1.2677103382701062e-05,0.86,0.58
1,5.531466853987899e-07,0.58,0.54
2,3.2509931250334833e-07,0.44,0.72
3,5.756860229156435e-09,0.50,0.72
4,1.181993501685605e-10,0.72,0.70
5,6.077865387430382e-13,0.68,0.60
6,1.2594535722773976e-14,0.54,0.76
7,2.9533344279101876e-16,0.54,0.76
8,1.3106722086846295e-17,0.76,0.78
9,8.516662136777213e-20,0.62,0.82
